# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Bank_account').getOrCreate()

In [10]:
df = spark.read.csv('customer_churn.csv' , header=True , inferSchema=True)

In [11]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [12]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols= ['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'] , outputCol= 'features')

In [15]:
output_data = assembler.transform(df)

In [16]:
final_data = output_data.select('features','churn')

## Train split data 

In [17]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.classification import LogisticRegression

In [19]:
# Creamos el modelo de Ml
lr_model = LogisticRegression(labelCol='churn')

In [20]:
#entrenamos el modelo de ML 
lr_model_fitted = lr_model.fit(train_churn)

## Evaluate

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
predictions = lr_model_fitted.evaluate(test_churn)

In [24]:
predictions.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.46424739511484...|[0.98861769108259...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.14782094197642...|[0.99786642605139...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.29481868481610...|[0.99500752581296...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.35468227011671...|[0.79489406788201...|       0.0|
|[28.0,11204.23,0....|    0|[1.50431241628231...|[0.81821677714687...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.55970259704770...|[0.98964321467455...|       0.0|
|[29.0,12711.15,0....|    0|[5.17824730072899...|[0.99439372821676...|       0.0|
|[29.0,13240.01,1....|    0|[6.51314895021847...|[0.99851839845372...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.12367048740776...|[0.98407278352454...|       0.0|
|[30.0,12788.37,

In [25]:
predictions.accuracy

0.9128787878787878

In [26]:
predictions.areaUnderROC

0.9212353993475727

## CArgamos la nueva data

In [29]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [30]:
new_customers_assembled = assembler.transform(new_customers)

In [31]:
new_data_predictions = lr_model_fitted.transform(new_customers_assembled)

In [32]:
new_data_predictions.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

